# NDIF Monitor - Generation

**Model:** `EleutherAI/gpt-j-6b`

This notebook tests nnsight + NDIF functionality. Generated by [NDIF Monitor](https://github.com/davidbau/ndif-monitor).

## Setup

First, configure your API keys in Colab Secrets (🔑 icon in left sidebar):
- `NDIF_API`: Your NDIF API key from [nnsight.net](https://nnsight.net)
- `HF_TOKEN`: Your HuggingFace token (for gated models)


In [ ]:
# Install dependencies
!pip install -q nnsight torch


In [ ]:
# Configure authentication from Colab secrets
import os

try:
    from google.colab import userdata
    NDIF_API = userdata.get('NDIF_API')
    HF_TOKEN = userdata.get('HF_TOKEN')
    print('Loaded secrets from Colab')
except:
    # Fallback for local testing
    NDIF_API = os.environ.get('NDIF_API')
    HF_TOKEN = os.environ.get('HF_TOKEN')
    print('Using environment variables')

# Configure nnsight
from nnsight import CONFIG
if NDIF_API:
    CONFIG.set_default_api_key(NDIF_API)
    print('NDIF API key configured')
else:
    print('Warning: NDIF_API not set - add it to Colab Secrets')

if HF_TOKEN:
    os.environ['HF_TOKEN'] = HF_TOKEN
    print('HuggingFace token configured')


In [ ]:
# Load model
from nnsight import LanguageModel
import time

MODEL_NAME = 'EleutherAI/gpt-j-6b'
print(f'Loading {MODEL_NAME}...')

start = time.time()
model = LanguageModel(MODEL_NAME, device_map='auto')
load_time = time.time() - start
print(f'Model loaded in {load_time:.1f}s')


## Generation Test

Tests `model.generate()` functionality.


In [ ]:
# Run generation
prompt = 'Once upon a time, there was a'
print(f"Generating from: '{prompt}'")

start = time.time()
with model.generate(prompt, max_new_tokens=20, remote=True):
    output_ids = model.generator.output.save()

gen_time = time.time() - start

# Decode output
generated_text = model.tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f'Generation completed in {gen_time:.1f}s')
print(f'Generated: {generated_text}')


In [ ]:
# Validate generation
if 'generated_text' not in dir():
    raise RuntimeError('Generation was interrupted - output not captured. Try running again.')

assert len(generated_text) > len(prompt), 'No text was generated'
assert generated_text.startswith(prompt[:20]), 'Generated text does not start with prompt'

print('Validation passed!')


In [ ]:
print('\n' + '=' * 50)
print('GENERATION TEST PASSED ✓')
print('=' * 50)
